# Imports

In [29]:
import pandas as pd
import json
import re
from transformers import AutoTokenizer
pd.set_option('display.max_colwidth', None)

# Functions

In [30]:
def read_file_txt(file_path)->str:

  with open(file_path, "r", encoding="Windows-1252", errors="ignore") as file:
    content = file.read()

  return content

def count_ids(content: str) -> None:
    """
    Conta IDs com sufixos específicos (ins, inp, out) no conteúdo fornecido.

    Args:
        content (str): O texto que contém os IDs a serem contados.

    Prints:
        Contagem de IDs por sufixo e o total de IDs.
    """
    suffixes = ["Q1", "C1", "C2", 'C3', "C4", "C5", "C6", 'C7', "C8", "C9", "C10"]
    counts = {}

    for suffix in suffixes:
        pattern = fr'[\'"]ID[\'"]:\s*[\'"]\d+{suffix}[\'"]' #     pattern = fr'"ID":\s*"\d+{suffix}"' pega menos id
        matches = re.findall(pattern, content)
        counts[suffix] = len(matches)
        print(f"Total de IDs com sufixo '{suffix}': {counts[suffix]}")

    total_ids = sum(counts.values())
    print("Total ids:", total_ids)

def extrair_dados_multiplos(texto):
    """
    Extrai dados estruturados de um texto baseado em um padrão de expressão regular,
    considerando aspas simples ou duplas nos campos.
    """
    # Padrão ajustado para aceitar aspas simples ou duplas
    padrao = re.compile(
        r"[\'\"]ID[\'\"]:\s*[\'\"]([^\'\"]+)[\'\"]\s*,\s*[\'\"]ORIGINAL[\'\"]:\s*[\'\"](.*?)(?<!\\)[\'\"]\s*,\s*[\'\"]FIGURATIVO[\'\"]:\s*(null|[\'\"](.*?)(?<!\\)[\'\"])\s*,\s*[\'\"]REESCRITO[\'\"]:\s*(null|[\'\"](.*?)(?<!\\)[\'\"])\s*,\s*[\'\"]RESULTADO[\'\"]:\s*[\'\"](.*?)(?<!\\)[\'\"]",
        re.DOTALL
    )

    matches = padrao.findall(texto)
    resultados = []

    for match in matches:
        id_val = match[0]
        original_val = match[1]
        figurativo_val = match[3] if match[2] != 'null' else None
        reescrito_val = match[5] if match[4] != 'null' else None
        resultado_val = match[6]

        resultados.append((id_val, original_val, figurativo_val, reescrito_val, resultado_val))

    return resultados

def processar_content_para_dataframe(content):

    ids, originais, figurativos, reescritos, resultados = [], [], [], [], []

    blocos_dados = content.split("json\n[\n")
    blocos_dados = [bloco.replace('\u200b', '').strip() for bloco in blocos_dados]

    for bloco in blocos_dados:
        conjuntos = extrair_dados_multiplos(bloco)
        for id_val, original_val, figurativo_val, reescrito_val, resultado_val in conjuntos:
            if id_val and original_val and resultado_val:
                ids.append(id_val)
                originais.append(original_val)
                figurativos.append(figurativo_val)
                reescritos.append(reescrito_val)
                resultados.append(resultado_val)

    df_result = pd.DataFrame({
        'ID': ids,
        'ORIGINAL': originais,
        'FIGURATIVO': figurativos,
        'REESCRITO': reescritos,
        'RESULTADO': resultados
    })

    print(f"DataFrame - shape: {df_result.shape}, IDs únicos: {df_result.ID.nunique()}")

    return df_result

def drop_duplicates_by_count_col(df, col):

  df['COUNT_'+col]= df[col].apply(lambda x: None if x==None else len(x))
  df = df.sort_values(by='COUNT_'+col, ascending=False)
  df = df.drop_duplicates(subset='ID', keep='first')
  df = df.sort_values(by='ID', ascending=True)

  print(f"After drop\nDataFrame - shape: {df.shape}, IDs únicos: {df.ID.nunique()}")

  return df

def process_dataframe_to_base(df, ins_column, inp_column, out_column):
    """
    Processa um DataFrame para criar um novo DataFrame base com as colunas especificadas.

    Parâmetros:
        df (pd.DataFrame): DataFrame original.
        ins_column (str): Nome da coluna para 'INSTRUCTION'.
        inp_column (str): Nome da coluna para 'INPUT'.
        out_column (str): Nome da coluna para 'OUTPUT'.

    Retorna:
        pd.DataFrame: DataFrame base com colunas 'ID', 'INSTANCE', e 'CLASS_TYPE'.
    """
    df_base = df.copy()

    # Criando o DataFrame ins
    ins = df_base[['ID', ins_column]].copy()
    ins['CLASS_TYPE'] = 'ins'
    ins['ID'] = ins['ID'].astype(str) + ins['CLASS_TYPE']
    ins = ins.rename(columns={ins_column: 'INSTANCE'})

    # Criando o DataFrame inp
    inp = df_base[['ID', inp_column]].copy()
    inp['CLASS_TYPE'] = 'inp'
    inp['ID'] = inp['ID'].astype(str) + inp['CLASS_TYPE']
    inp = inp.rename(columns={inp_column: 'INSTANCE'})

    # Criando o DataFrame out
    out = df_base[['ID', out_column]].copy()
    out['CLASS_TYPE'] = 'out'
    out['ID'] = out['ID'].astype(str) + out['CLASS_TYPE']
    out = out.rename(columns={out_column: 'INSTANCE'})

    # Concatenando os DataFrames
    df_base = pd.concat([ins, inp, out], axis=0)[['ID', 'INSTANCE', 'CLASS_TYPE']]

    return df_base

# Base de Dados

In [31]:
mmlu = '../DATASETS/mmlu_to_figurative.csv'

mmlu_pro_base = '../DATASETS/mmlu_pro_to_figurative.csv'
mmlu_pro_bucket1 = '../DATASETS/MMLU_PRO/figurative_mmlu_pro_to_figurative_response.txt'
mmlu_pro_bucket2 = '../DATASETS/MMLU_PRO/figurative_mmlu_pro_bucket_to_figurative_again_response.txt'
mmlu_pro_single = '../DATASETS/MMLU_PRO/figurative_mmlu_pro_to_figurative_single_response.txt'

## Base

In [32]:
df_base = pd.read_csv(mmlu_pro_base)
df_base.head(2)

,ID,ORIGINAL,TOKENS_EST_ID,TOKENS_EST_ORIGINAL,TOTAL_TOKENS
0,0Q1,"O grupo simétrico $S_n$ tem $\n\factorial{n}$ elementos, portanto não é verdade que $S_{10}$ tem 10 elementos.\nEncontre a característica do anel 2Z.",3,61,64
1,1Q1,"Seja V o conjunto de todos os polinômios reais p(x). Sejam as transformações T, S definidas em V por T:p(x) -> xp(x) e S:p(x) -> p'(x) = d/dx p(x), e interprete (ST)(p(x)) como S(T(p(x))). Qual das seguintes opções é verdadeira?",3,110,113


In [33]:
print("Bucket---")
content_bucket = read_file_txt(mmlu_pro_bucket1)
count_ids(content_bucket)
df_bucket = processar_content_para_dataframe(content_bucket)
df_bucket.query("ID =='56Q1'")

Bucket---
Total de IDs com sufixo 'Q1': 8232
Total de IDs com sufixo 'C1': 3974
Total de IDs com sufixo 'C2': 4180
Total de IDs com sufixo 'C3': 2704
Total de IDs com sufixo 'C4': 0
Total de IDs com sufixo 'C5': 0
Total de IDs com sufixo 'C6': 0
Total de IDs com sufixo 'C7': 0
Total de IDs com sufixo 'C8': 0
Total de IDs com sufixo 'C9': 0
Total de IDs com sufixo 'C10': 0
Total ids: 19090
DataFrame - shape: (19058, 5), IDs únicos: 19039


,ID,ORIGINAL,FIGURATIVO,REESCRITO,RESULTADO
25,56Q1,"Você me fará a justiça de lembrar que sempre apoiei arduamente o Direito de cada homem à sua própria opinião, por mais diferente que essa opinião possa ser da minha. Aquele que nega a outro esse direito, torna-se escravo de sua opinião atual, porque se impede do direito de mudá-la.",escravo de sua opinião atual,prisioneiro de sua própria crença,"Você me fará a justiça de lembrar que sempre apoiei arduamente o Direito de cada homem à sua própria opinião, por mais diferente que essa opinião possa ser da minha. Aquele que nega a outro esse direito, torna-se prisioneiro de sua própria crença, porque se impede do direito de mudá-la."
26,56Q1,A arma mais formidável contra erros de todo tipo é a Razão. Nunca usei nenhuma outra e confio que nunca usarei.,A arma mais formidável contra erros de todo tipo é a Razão,A ferramenta mais eficaz contra erros de todo tipo é a Razão,A ferramenta mais eficaz contra erros de todo tipo é a Razão. Nunca usei nenhuma outra e confio que nunca usarei.
27,56Q1,"A circunstância que agora ocorreu na França da abolição total de toda a ordem nacional do sacerdócio e de tudo o que pertence a sistemas compulsivos de religião e artigos compulsivos de fé não apenas precipitou minha intenção, mas tornou uma obra desse tipo extremamente necessária, para que no naufrágio geral da superstição, de falsos sistemas de governo e falsa teologia, não percamos de vista a moralidade, a humanidade e a teologia que é verdadeira.","naufragio geral da superstição, de falsos sistemas de governo e falsa teologia","colapso geral da superstição, de falsos sistemas de governo e falsa teologia","A circunstância que agora ocorreu na França da abolição total de toda a ordem nacional do sacerdócio e de tudo o que pertence a sistemas compulsivos de religião e artigos compulsivos de fé não apenas precipitou minha intenção, mas tornou uma obra desse tipo extremamente necessária, para que no colapso geral da superstição, de falsos sistemas de governo e falsa teologia, não percamos de vista a moralidade, a humanidade e a teologia que é verdadeira."
28,56Q1,Minha própria mente é minha própria igreja.,Minha própria mente é minha própria igreja.,Minha própria consciência é minha própria guia.,Minha própria consciência é minha própria guia.
29,56Q1,"Todas as instituições nacionais de igrejas, sejam judaicas, cristãs ou turcas, me parecem nada mais do que invenções humanas, criadas para aterrorizar e escravizar a humanidade e monopolizar o poder e o lucro.","invenções humanas, criadas para aterrorizar e escravizar a humanidade e monopolizar o poder e o lucro","criações humanas, feitas para oprimir e controlar a humanidade e acumular poder e riqueza","Todas as instituições nacionais de igrejas, sejam judaicas, cristãs ou turcas, me parecem nada mais do que criações humanas, feitas para oprimir e controlar a humanidade e acumular poder e riqueza."


In [34]:
print("Bucket---")
content_bucket = read_file_txt(mmlu_pro_bucket2)
count_ids(content_bucket)
df_bucket2 = processar_content_para_dataframe(content_bucket)
df_bucket2.head()

Bucket---
Total de IDs com sufixo 'Q1': 1067
Total de IDs com sufixo 'C1': 1657
Total de IDs com sufixo 'C2': 1322
Total de IDs com sufixo 'C3': 2490
Total de IDs com sufixo 'C4': 4166
Total de IDs com sufixo 'C5': 3710
Total de IDs com sufixo 'C6': 3707
Total de IDs com sufixo 'C7': 3749
Total de IDs com sufixo 'C8': 3893
Total de IDs com sufixo 'C9': 3330
Total de IDs com sufixo 'C10': 2844
Total ids: 31935
DataFrame - shape: (31127, 5), IDs únicos: 31112


,ID,ORIGINAL,FIGURATIVO,REESCRITO,RESULTADO
0,25Q1,"Preveja o número de linhas no espectro EPR de uma solução de radical metil marcado com 13C (13CH3•), supondo que as linhas não se sobreponham.",None,None,"Preveja o número de linhas no espectro EPR de uma solução de radical metil marcado com 13C (13CH3•), supondo que as linhas não se sobreponham."
1,26Q1,"Qual das opções a seguir lista os hidretos dos elementos do grupo 14 em ordem de estabilidade térmica, do menor para o maior?",None,None,"Qual das opções a seguir lista os hidretos dos elementos do grupo 14 em ordem de estabilidade térmica, do menor para o maior?"
2,27Q1,Qual dos seguintes é considerado um anidrido ácido?,None,None,Qual dos seguintes é considerado um anidrido ácido?
3,28Q1,"Um novo composto é sintetizado e descoberto como sendo um ácido monoprótico com uma massa molar de 248 g/mol. Quando 0,0050 mol deste ácido são dissolvidos em 0,500 L de água, o pH é medido como 3,89. Qual é o pKa deste ácido?",None,None,"Um novo composto é sintetizado e descoberto como sendo um ácido monoprótico com uma massa molar de 248 g/mol. Quando 0,0050 mol deste ácido são dissolvidos em 0,500 L de água, o pH é medido como 3,89. Qual é o pKa deste ácido?"
4,29Q1,"Uma solução contém 2,00 mol de ácido acético, CH3COOH, e 1,00 mol de acetato de cálcio, Ca(CH3COO)2. A solução é capaz de resistir à adição de uma pequena quantidade de ácido forte ou base forte com apenas pequenas alterações no pH da solução. Quantidades maiores de ácido forte ou base forte podem causar uma alteração significativa no pH. Quantos mols de ácido nítrico, HNO3, podem ser adicionados antes que o pH comece a mudar significativamente?",resistível à adição de uma pequena quantidade de ácido forte ou base forte com apenas pequenas alterações no pH da solução,capaz de suportar a adição de uma pequena quantidade de ácido forte ou base forte com pequenas variações no pH da solução,"Uma solução contém 2,00 mol de ácido acético, CH3COOH, e 1,00 mol de acetato de cálcio, Ca(CH3COO)2. A solução é capaz de suportar a adição de uma pequena quantidade de ácido forte ou base forte com pequenas variações no pH da solução. Quantidades maiores de ácido forte ou base forte podem causar uma alteração significativa no pH. Quantos mols de ácido nítrico, HNO3, podem ser adicionados antes que o pH comece a mudar significativamente?"


In [35]:
bucket_complete = pd.concat([df_bucket, df_bucket2])
bucket_complete.shape

(50185, 5)

In [36]:
df_bucket.shape[0]+df_bucket2.shape[0]

50185

## Single

In [38]:
print("Single---")
content_single = read_file_txt("../DATASETS/MMLU_PRO/figurative_mmlu_pro_to_figurative_single_response.txt")
count_ids(content_single)
df_single = processar_content_para_dataframe(content_single)
#df_single = drop_duplicates_by_count_col(df_single, 'RESULTADO')
df_single.head()

Single---
Total de IDs com sufixo 'Q1': 2608
Total de IDs com sufixo 'C1': 6306
Total de IDs com sufixo 'C2': 6432
Total de IDs com sufixo 'C3': 6746
Total de IDs com sufixo 'C4': 7771
Total de IDs com sufixo 'C5': 7778
Total de IDs com sufixo 'C6': 7711
Total de IDs com sufixo 'C7': 7597
Total de IDs com sufixo 'C8': 7364
Total de IDs com sufixo 'C9': 7565
Total de IDs com sufixo 'C10': 7561
Total ids: 75439
DataFrame - shape: (74616, 5), IDs únicos: 74539


,ID,ORIGINAL,FIGURATIVO,REESCRITO,RESULTADO
0,41Q1,"A bobina de um medidor de bobina móvel tem 100 voltas, 40 mm de comprimento e 30 mm de largura. O torque de controle é 240*10-6 N-m em escala total. Se a densidade do fluxo magnético for 1Wb/m2, a faixa do medidor é",None,None,"A bobina de um medidor de bobina móvel tem 100 voltas, 40 mm de comprimento e 30 mm de largura. O torque de controle é 240*10-6 N-m em escala total. Se a densidade do fluxo magnético for 1Wb/m2, a faixa do medidor é"
1,42Q1,"Em uma trava SR construída a partir de portas NOR, qual condição não é permitida?",None,None,"Em uma trava SR construída a partir de portas NOR, qual condição não é permitida?"
2,43Q1,"Dois condutores longos paralelos transportam 100 A. Se os condutores estiverem separados por 20 mm, a força por metro de comprimento de cada condutor será",None,None,"Dois condutores longos paralelos transportam 100 A. Se os condutores estiverem separados por 20 mm, a força por metro de comprimento de cada condutor será"
3,45Q1,"Qual das fórmulas dadas de PL é a melhor simbolização da seguinte frase?\nAs tartarugas vivem vidas longas e são criaturas felizes, a menos que sejam feridas.",a menos que sejam feridas,exceto se forem feridas,"Qual das fórmulas dadas de PL é a melhor simbolização da seguinte frase?\nAs tartarugas vivem vidas longas e são criaturas felizes, exceto se forem feridas."
4,46Q1,Selecione a melhor tradução para lógica de predicados. George pega emprestado o cortador de grama de Hector. (g: George; h: Hector; l: cortador de grama de Hector; Bxyx: x pega emprestado y de z).,George pega emprestado o cortador de grama de Hector,George recebe o cortador de grama de Hector temporariamente,Selecione a melhor tradução para lógica de predicados. George recebe o cortador de grama de Hector temporariamente. (g: George; h: Hector; l: cortador de grama de Hector; Bxyx: x pega emprestado y de z).


# Join MMLU PRO RESULTS FIGURATIVE

In [23]:
df_to_gloss = pd.concat([ bucket_complete,df_single])
df_to_gloss.head()

,ID,ORIGINAL,FIGURATIVO,REESCRITO,RESULTADO
0,0Q1,"O grupo simétrico $S_n$ tem $\n\\factorial{n}$ elementos, portanto não é verdade que $S_{10}$ tem 10 elementos.\nEncontre a característica do anel 2Z.",portanto não é verdade que $S_{10}$ tem 10 elementos,"portanto, é falso que $S_{10}$ tenha 10 elementos","O grupo simétrico $S_n$ tem $\n\\factorial{n}$ elementos, portanto, é falso que $S_{10}$ tenha 10 elementos.\nEncontre a característica do anel 2Z."
1,1Q1,"Seja V o conjunto de todos os polinômios reais p(x). Sejam as transformações T, S definidas em V por T:p(x) -> xp(x) e S:p(x) -> p'(x) = d/dx p(x), e interprete (ST)(p(x)) como S(T(p(x))). Qual das seguintes opções é verdadeira?",None,None,"Seja V o conjunto de todos os polinômios reais p(x). Sejam as transformações T, S definidas em V por T:p(x) -> xp(x) e S:p(x) -> p"
2,2Q1,"Seja A o conjunto de todos os pares ordenados de inteiros (m, n) tais que 7m + 12n = 22. Qual é o maior número negativo no conjunto B = {m + n : (m, n) \\em A}?",None,None,"Seja A o conjunto de todos os pares ordenados de inteiros (m, n) tais que 7m + 12n = 22. Qual é o maior número negativo no conjunto B = {m + n : (m, n) \\em A}?"
3,3Q1,"Um tanque contém inicialmente uma solução salina de 3 gramas de sal dissolvido em 100 litros de água. Uma solução salina contendo 0,02 gramas de sal por litro de água é pulverizada no tanque a uma taxa de 4 litros por minuto. A solução pulverizada é continuamente misturada com a solução salina no tanque, e a mistura flui para fora do tanque a uma taxa de 4 litros por minuto. Se a mistura for instantânea, quantos gramas de sal estão no tanque após 100 minutos?",None,None,"Um tanque contém inicialmente uma solução salina de 3 gramas de sal dissolvido em 100 litros de água. Uma solução salina contendo 0,02 gramas de sal por litro de água é pulverizada no tanque a uma taxa de 4 litros por minuto. A solução pulverizada é continuamente misturada com a solução salina no tanque, e a mistura flui para fora do tanque a uma taxa de 4 litros por minuto. Se a mistura for instantânea, quantos gramas de sal estão no tanque após 100 minutos?"
4,4Q1,Um total de 30 jogadores jogarão basquete em um parque. Haverá exatamente 5 jogadores em cada time. Qual afirmação explica corretamente como encontrar o número de times necessários?,None,None,Um total de 30 jogadores jogarão basquete em um parque. Haverá exatamente 5 jogadores em cada time. Qual afirmação explica corretamente como encontrar o número de times necessários?


# Format to Gloss

In [24]:
def calculate_tokens(df, columns, tokenizer_model="gpt2"):
    """
    Calcula o número de tokens para colunas especificadas em um DataFrame usando um tokenizer.

    Args:
        df (pd.DataFrame): DataFrame de entrada contendo os dados de texto.
        columns (list): Lista de colunas do DataFrame para calcular os tokens.
        tokenizer_model (str): Nome do modelo para carregar o tokenizer. Padrão é "gpt2".

    Returns:
        pd.DataFrame: DataFrame atualizado com colunas contendo o número de tokens.
    """
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_model)

    def count_tokens(text):
        if not text or not isinstance(text, str):
            return 0
        return len(tokenizer.encode(text, truncation=True))

    for column in columns:
        token_column_name = f"TOKENS_EST_{column.upper()}"
        df[token_column_name] = df[column].apply(count_tokens)

    total_tokens_column = "TOTAL_TOKENS"
    token_columns = [f"TOKENS_EST_{col.upper()}" for col in columns if f"TOKENS_EST_{col.upper()}" in df.columns]

    if token_columns:
        df[total_tokens_column] = df[token_columns].sum(axis=1)
    else:
        df[total_tokens_column] = 0
    return df

In [25]:
df_to_gloss = df_to_gloss[['ID', 'RESULTADO']].rename(columns={'RESULTADO':'PT'})
df_to_gloss.head()

,ID,PT
0,0Q1,"O grupo simétrico $S_n$ tem $\n\\factorial{n}$ elementos, portanto, é falso que $S_{10}$ tenha 10 elementos.\nEncontre a característica do anel 2Z."
1,1Q1,"Seja V o conjunto de todos os polinômios reais p(x). Sejam as transformações T, S definidas em V por T:p(x) -> xp(x) e S:p(x) -> p"
2,2Q1,"Seja A o conjunto de todos os pares ordenados de inteiros (m, n) tais que 7m + 12n = 22. Qual é o maior número negativo no conjunto B = {m + n : (m, n) \\em A}?"
3,3Q1,"Um tanque contém inicialmente uma solução salina de 3 gramas de sal dissolvido em 100 litros de água. Uma solução salina contendo 0,02 gramas de sal por litro de água é pulverizada no tanque a uma taxa de 4 litros por minuto. A solução pulverizada é continuamente misturada com a solução salina no tanque, e a mistura flui para fora do tanque a uma taxa de 4 litros por minuto. Se a mistura for instantânea, quantos gramas de sal estão no tanque após 100 minutos?"
4,4Q1,Um total de 30 jogadores jogarão basquete em um parque. Haverá exatamente 5 jogadores em cada time. Qual afirmação explica corretamente como encontrar o número de times necessários?


In [26]:
df_to_gloss_final = calculate_tokens(df_to_gloss, ['ID', 'PT'])
print(f"To gloss:{df_to_gloss_final.shape}, ids únicos: {df_to_gloss_final['ID'].nunique()}")
df_to_gloss_final.head()

To gloss:(124801, 5), ids únicos: 124688


,ID,PT,TOKENS_EST_ID,TOKENS_EST_PT,TOTAL_TOKENS
0,0Q1,"O grupo simétrico $S_n$ tem $\n\\factorial{n}$ elementos, portanto, é falso que $S_{10}$ tenha 10 elementos.\nEncontre a característica do anel 2Z.",3,61,64
1,1Q1,"Seja V o conjunto de todos os polinômios reais p(x). Sejam as transformações T, S definidas em V por T:p(x) -> xp(x) e S:p(x) -> p",3,59,62
2,2Q1,"Seja A o conjunto de todos os pares ordenados de inteiros (m, n) tais que 7m + 12n = 22. Qual é o maior número negativo no conjunto B = {m + n : (m, n) \\em A}?",3,69,72
3,3Q1,"Um tanque contém inicialmente uma solução salina de 3 gramas de sal dissolvido em 100 litros de água. Uma solução salina contendo 0,02 gramas de sal por litro de água é pulverizada no tanque a uma taxa de 4 litros por minuto. A solução pulverizada é continuamente misturada com a solução salina no tanque, e a mistura flui para fora do tanque a uma taxa de 4 litros por minuto. Se a mistura for instantânea, quantos gramas de sal estão no tanque após 100 minutos?",3,165,168
4,4Q1,Um total de 30 jogadores jogarão basquete em um parque. Haverá exatamente 5 jogadores em cada time. Qual afirmação explica corretamente como encontrar o número de times necessários?,3,63,66


In [27]:
df_to_gloss_final.to_csv("../DATASETS/mmlu_pro_to_gloss.csv", index=False)